In [1]:
#export
import math,torch,matplotlib.pyplot as plt
import fastcore.all as fc
from collections.abc import Mapping
from operator import attrgetter
from functools import partial
from copy import copy

from torch import optim
import torch.nn.functional as F

try:
    from .conv import *
except:
    from src.miniai.conv import *

from fastprogress import progress_bar,master_bar

In [11]:
import matplotlib as mpl
import torchvision.transforms.functional as TF
from contextlib import contextmanager
from torch import nn,tensor
from datasets import load_dataset,load_dataset_builder

try:
    from .datasets import *
    from .conv import *
except:
    from src.miniai.datasets import *
    from src.miniai.conv import *
import logging
from fastcore.test import test_close

from torch.utils.data import DataLoader

In [12]:
# Using our dataset from before
# we will be using a dataset dictionary from huggingface
x, y = 'image', 'label'
name = "fashion_mnist"
dsd = load_dataset(name)

Reusing dataset fashion_mnist (/Users/diegomedina-bernal/.cache/huggingface/datasets/fashion_mnist/fashion_mnist/1.0.0/8d6c32399aa01613d96e2cbc9b13638f359ef62bb33612b077b4c247f6ef99c1)
100%|██████████| 2/2 [00:00<00:00, 707.42it/s]


In [13]:
@inplace
def transformi(b):
    b[x] = [torch.flatten(TF.to_tensor(o)) for o in b[x]]

In [14]:
bs = 256
tds = dsd.with_transform(transformi) # transformi is an inplace function

In [15]:
dsd['train']['image'][0]

In [16]:
tds['train']['image'][0].shape

torch.Size([784])

In [22]:
#export
class DataLoaders:
    """This class is a wrapper for the dataloaders, it specifically takes in a dataset dictionary from huggingface in this case"""
    def __init__(self, *dls):
        self.train, self.valid = dls[:2]

    @classmethod
    def from_dd(cls, dd, batch_size, num_workers=0, as_tuple=False):
        """a dd is a dataset dictionary from huggingface. calling values return the dictionary values which are the datasets in the object"""
        return cls(*[DataLoader(ds, batch_size, num_workers=num_workers, collate_fn=collate_dict(ds)) for ds in dd.values()])

In [23]:
dls = DataLoaders.from_dd(tds, bs, num_workers=4)
dt = dls.train
xb, yb = next(iter(dt))
xb.shape, yb[:10]

(torch.Size([256, 784]), tensor([9, 0, 0, 3, 0, 2, 7, 2, 5, 5]))

In [45]:
#export
class Learner:
    """This is our main class that will be responsible to experimenting and training various models. It serves the purpose of being a vehicle to experiment quickly"""
    def __init__(self, model, dls, loss_func, lr, opt_func=optim.SGD):
        """
        model: <PyTorch Model> the model we are going to train
        dls: <Pytorch DataLoader> the dataloaders we are going to use
        loss_func: <Callable> the loss function we are going to use
        lr: <float> the learning rate we are going to use
        opt_func: <PyTorch optimizer (can be custom)> the optimizer we are going to use
        """
        fc.store_attr() # easy method to store all attributes, prevents mistakes

    def one_batch(self):
        """This method is responsible for one batch of training/evaluation"""
        self.xb, self.yb = to_device(self.batch) # move batch to device
        self.preds = self.model(self.xb) # get predictions
        self.loss = self.loss_func(self.preds, self.yb) # calculate loss
        if self.model.training:
            self.loss.backward() # calculate gradients
            self.opt.step() # update weights
            self.opt.zero_grad() # zero gradients
        with torch.no_grad(): self.calc_stats() # calculate stats
    
    def calc_stats(self):
        """This method is responsible for calculating stats"""
        acc = (self.preds.argmax(dim=1)==self.yb).float().sum() # calculate accuracy
        self.accs.append(acc) # append accuracy to list
        n = len(self.xb)
        self.losses.append(self.loss*n) # append loss to list
        self.ns.append(n) # append number of samples to list
    
    def one_epoch(self, train):
        self.model.training = train # set model to training mode or evaluation mode
        dl = self.dls.train if train else self.dls.valid # get the correct dataloader
        for self.num, self.batch in enumerate(dl): 
            self.one_batch() # call one_batch method, self.batch will be set to the current batch
        n = sum(self.ns)
        print(f"Epoch: {self.epoch+1}", 'train' if self.model.training else 'valid', f"Loss: {sum(self.losses).item()/n}", f"Accuracy: {sum(self.accs).item()/n*100}%")
    
    def fit(self, n_epochs):
        """Very lightweight fit method, this is the highest level to train a model via the Learner object"""
        self.accs, self.losses, self.ns = [], [], []
        self.model.to(def_device) # place model on appropriate device TODO: Allow this to be changed via fit calls if necessary
        self.opt = self.opt_func(self.model.parameters(), lr=self.lr) # create optimizer
        self.n_epochs = n_epochs
        for self.epoch in range(n_epochs):
            self.one_epoch(True) # training
            self.one_epoch(False) # evaluation

In [46]:
# creating a simple model to test
m, nh = 28*28, 50
model = nn.Sequential(
    nn.Linear(m, nh),
    nn.ReLU(),
    nn.Linear(nh, 10) # output
)

In [47]:
# creating our learner to test
learn = Learner(model, dls, F.cross_entropy, lr=0.2)
learn.fit(1)

Epoch: 1 train Loss: 0.774168 Accuracy: 72.62166666666666%
Epoch: 1 valid Loss: 0.767629 Accuracy: 72.92285714285714%


### Metric
Our metric class will be responsible for calculating any metrics/calculations for the model for performance

In [48]:
#export
class Metric:
    def __init__(self): self.reset()
    def reset(self): self.vals, self.ns = [], [] # reset values and number of samples
    def add(self, inp, targ=None, n=1):
        """
        inp: <Tensor> the input tensor
        targ: <Tensor> the target tensor
        n: <int> the number of samples, the minibatch size is another way to interpret this
        """
        self.last = self.calc(inp, targ) # calculate the metric, this will change based on the metric
        self.vals.append(self.last) # append the last metric to the list
        self.ns.append(n) # append the number of samples to the list
    @property
    def value(self):
        """This property is responsible for calculating the value of the metric
        It will also normalize the metric by the number of samples returning the average
        """
        ns = tensor(self.ns)
        return (tensor(self.vals)*ns).sum()/ns.sum()
    def calc(self, inps, targs): return inps

In [49]:
#export
class Accuracy(Metric):
    def calc(self, inps, targs): return (inps==targs).float().mean()

In [51]:
acc = Accuracy()
acc.add(tensor([0,1,2,0,1,2]), tensor([0,1,1,2,1,0])) # adding a batch or example
acc.add(tensor([1,1,2,0,1]), tensor([0,1,1,2,1])) # adding a batch or example, this contains less values but will still work
acc.value

tensor(0.4500)

In [52]:
loss = Metric() # loss can be used as a metric
loss.add(0.6, n=32) # adding a batch of 32 examples
loss.add(0.9, n=2) # adding a batch of 2 examples
loss.value

tensor(0.6176)

In [53]:
(0.6*32 + 0.9*2)/(32+2)

0.6176470588235294

## Callbacks

In [ ]:
# TODO: Finish lecture 15 and really go through the code
# TODO: Try to implement a custom callback
# TODO: Start lecture 16
# TODO: Recreate a Learner class with custom callbacks from scratch. This project alone should take a few days
# -> For this project try to make a learner that is good with text data -> Implement huggingface transformers for now
# This will require learning about different transformer models and how to use them with their individual tokenizers
# I should create/test for text classification first, then move on to text generation. The idea here is i'm going to be fine-tuning these bigger models
# TODO: Extra: Experiment with optimization functions.